In [1]:
from PIL import Image
import requests
import torch
from transformers import AutoProcessor
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor

from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"

dataset = load_dataset('eduvedras/Img_Desc',split='test',trust_remote_code=True)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.0) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/eduvedras/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def join_predictions(template,vars):
    if 'decision tree with depth = 2' in template:
        variables = vars.split(',')
        first_index = 0
        second_index = 0
        for i in range(len(template)):
            if template[i] == '[' and first_index == 0:
                first_index = i
            elif template[i] == '[' and first_index != 0:
                second_index = i
                break
        return template[:first_index] + variables[0] + template[first_index+2:second_index] + variables[1][1:] + template[second_index+2:]
    elif 'A bar chart showing the explained variance ratio' in template or 'A bar chart showing the distribution of the target variable' in template:
        return template.replace('[]',vars)
    else:
        return template.replace('[]','['+vars+']')
                

In [3]:
from huggingface_hub import InferenceClient
from transformers import AutoModelForCausalLM

filetag = "pix2struct-textcaps-base-desc-final"

model_templates = Pix2StructForConditionalGeneration.from_pretrained("eduvedras/pix2struct-textcaps-base-desc-templates-final").to(device)
model_vars = Pix2StructForConditionalGeneration.from_pretrained("eduvedras/pix2struct-textcaps-base-desc-vars-final").to(device)

In [4]:
checkpoint = "google/pix2struct-textcaps-base"

processor = Pix2StructProcessor.from_pretrained(checkpoint)

predictions = []
references = []

dic = {}

from tqdm.auto import tqdm

for i in range(len(dataset)):
    dic[dataset[i]['Chart_name']] = []

for i in range(len(dataset)):
    dic[dataset[i]['Chart_name']].append(dataset[i]['Description'])
    
print(len(dic))

15


In [5]:
for i in tqdm(range(len(dataset))):
    inputs = processor(images=dataset[i]['Chart'], return_tensors="pt").to(device)
    flattened_patches = inputs.flattened_patches
    attention_mask = inputs.attention_mask
    generated_template_ids = model_templates.generate(flattened_patches=flattened_patches, attention_mask=attention_mask, max_length=193)
    generated_template = processor.batch_decode(generated_template_ids, skip_special_tokens=True)[0]  
    generated_vars_ids = model_vars.generate(flattened_patches=flattened_patches, attention_mask=attention_mask, max_length=167)
    generated_vars = processor.batch_decode(generated_vars_ids, skip_special_tokens=True)[0]
    generated_caption = join_predictions(generated_template,generated_vars)
    predictions.append(generated_caption)
    references.append(dic[dataset[i]['Chart_name']])

100%|██████████| 15/15 [00:25<00:00,  1.73s/it]


In [6]:
import evaluate


file = open(f"predictions-{filetag}.txt", "a")

bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
print(results)
file.write(f"BLEU: {results}\n")

meteor = evaluate.load("meteor")
results = meteor.compute(predictions=predictions, references=references)
print(results)
file.write(f"METEOR: {results}\n")

rouge = evaluate.load("rouge")
results = rouge.compute(predictions=predictions, references=references)
print(results)
file.write(f"ROUGE: {results}\n")
file.close()

{'bleu': 0.8543548665124875, 'precisions': [0.9277777777777778, 0.8608695652173913, 0.8272727272727273, 0.8063492063492064], 'brevity_penalty': 1.0, 'length_ratio': 1.0027855153203342, 'translation_length': 360, 'reference_length': 359}


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/eduvedras/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/eduvedras/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/eduvedras/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'meteor': 0.8806399322965363}
{'rouge1': 0.9757920924330462, 'rouge2': 0.9621711000182334, 'rougeL': 0.9759489302967564, 'rougeLsum': 0.9756877935840069}


In [7]:
import pandas as pd

new_df = pd.DataFrame(columns=['Image','Prediction'])

i=0
while i < len(dataset):
    if ((new_df['Image'] == dataset[i]['Chart_name']) & (new_df['Prediction'] == predictions[i])).any():
        i += 1
        continue
    else:
        new_df.loc[len(new_df)] = {'Image': dataset[i]['Chart_name'], 'Prediction': predictions[i]}
        i += 1
    
new_df.to_csv(f'predictions-{filetag}.csv', index=False)